### Essential climate variables for assessment of climate variability from 1979 to present

This Jupyter notebook will demonstrate how to access and visualise data from the Copernicus Climate Data Store (CDS) dataset **ERA5-Land hourly data from 1950 to present**. It will also show how the dataset's overview image and thumbnail image were generated.

For this demonstration, we will be using the Python package `earthkit` as a one-stop-shop for CDS data access, processing and visulisation. For more information on `earthkit`, including installation instructions, see its [GitHub Repository](https://github.com/ecmwf/earthkit).

In [30]:
import earthkit
from matplotlib.gridspec import GridSpec

#### Retrieving CDS data

Data from the CDS can be accessed through earthkit via the `earthkit.data.from_source()` function. The first argument should be the string `'cds'`, followed by the dataset name (slug), and finally a dictionary containing the contents of your request.

In [2]:
t_anom = earthkit.data.from_source(
    'cds',
    'ecv-for-climate-change',
    {
        'variable': 'surface_air_temperature',
        'product_type': 'anomaly',
        'climate_reference_period': '1991_2020',
        'time_aggregation': '1_month_mean',
        'year': '2023',
        'month': '06',
        'origin': 'era5',
    },
)

2023-08-03 09:54:34,535 INFO Welcome to the CDS
2023-08-03 09:54:34,535 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/ecv-for-climate-change
2023-08-03 09:54:34,667 INFO Request is queued
2023-08-03 09:54:35,721 INFO Request is running
2023-08-03 09:55:07,191 INFO Request is completed
2023-08-03 09:55:07,194 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data8/dataset-ecv-for-climate-change-00b14022-864a-4f50-ac6b-1bcb20421fc6.zip to /var/folders/vt/7j2c2tmx4m14gn_sg3zpf5l00000gn/T/earthkit-data-mavj/cds-retriever-19a9aa9a1a01ec9e181d919fa20bd704405aa420e78696349fcd09e4839674bc.cache.tmp (2.4M)
2023-08-03 09:55:07,721 INFO Download rate 4.6M/s                                                                                                                 


  0%|          | 0/1 [00:00<?, ?it/s]

#### Generating a simple image

This section will demonstrate how the overview image for this dataset was generated.

To start with, we need to define some *styles* for plotting our variables. For 2m temperature, we want a shaded contour style using the *turbo* colour palette from [matplotlib](https://matplotlib.org/stable/gallery/color/colormap_reference.html). Styles can also have *units* associated with them, so that any data plotted using that style will be converted to use the style's units.

In [27]:
import numpy as np

style = earthkit.maps.styles.Contour(
    colors="seismic",
    levels=np.arange(-11, 11.1, 0.1),
)

Now we can produce our plot, using the styles we just defined. We are using *cartopy* to specify a *Nearside Perspective* projection for our plot.

In [31]:
import cartopy.crs as ccrs

gs = GridSpec(1, 2, width_ratios=[2, 1])

chart = earthkit.maps.Superplot.from_gridspec(gs, rows=1, cols=2)

robinson = chart.add_subplot(crs=ccrs.Robinson())
robinson.plot(t_anom, style=style)

europe = chart.add_subplot(domain="Europe")
europe.plot(t_anom, style=style)

chart.coastlines(resolution="high")

chart.legend(location="right", label="°C", shrink=0.5, ticks=range(-10, 11, 2))

chart.title("Surface air temperature anomaly for {time:%B %Y}")

chart.show()

AttributeError: type object 'Superplot' has no attribute 'from_gridspec'